<a href="https://colab.research.google.com/github/soyoung33500/Sinkhole/blob/main/%EB%AA%A8%EB%8D%B8%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from math import *; import random; import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Dense, LeakyReLU
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/data/마지막최종.xlsx", index_col=0)

df_dummies = pd.get_dummies(df['지질'])

df2 = df.drop(['지질','규모'],axis=1)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(df2)
df_scaled1 = sc.transform(df2)
scd = pd.DataFrame(df_scaled1, columns = df2.columns)

all1 = pd.concat([df['규모'], scd, df_dummies], axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# min-max 정규화 적용
scaler.fit(df2)
df_scaled2 = scaler.transform(df2)
scd2 = pd.DataFrame(df_scaled2, columns = df2.columns)

all2 = pd.concat([df['규모'], scd2, df_dummies], axis=1)

In [ ]:
# 표준화
all1

,규모,traffic,하수도노후,총강수일,상수도노후,3km내공사건수,지하수이용량,지하철역수,BC11,BC29,BC36,기타
0,1,-0.457675,-0.825494,1.584589,0.635374,-1.243611,-0.697197,-1.257834,1,0,0,0
1,1,0.529829,-0.657399,2.838974,0.384043,0.499616,-0.504037,-0.601306,0,0,1,0
2,1,1.777957,0.840538,2.211781,0.007047,-0.662535,-0.165855,0.711750,0,1,0,0
3,1,2.123445,-1.790039,0.330204,1.012370,-0.081459,0.370026,2.024807,0,1,0,0
4,0,-0.573173,-1.790039,0.330204,1.012370,-1.243611,-0.354552,0.055222,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
102,1,1.271274,0.962711,-0.715117,1.640698,-0.662535,-0.445948,0.055222,0,1,0,0
103,0,-0.027823,0.213142,-0.715117,-1.626604,-1.243611,-0.900175,0.055222,0,0,1,0
104,0,-1.224730,1.238006,-0.715117,-0.746946,-1.243611,2.783289,-1.257834,0,1,0,0
105,0,5.179612,0.916046,-0.924181,-1.752270,-1.243611,0.155303,0.055222,1,0,0,0


In [ ]:
# 정규화
all2

,규모,traffic,하수도노후,총강수일,상수도노후,3km내공사건수,지하수이용량,지하철역수,BC11,BC29,BC36,기타
0,1,0.124849,0.286009,0.727273,0.542857,0.000000,0.067776,0.000000,1,0,0,0
1,1,0.278152,0.334818,1.000000,0.485714,0.500000,0.117463,0.166667,0,0,1,0
2,1,0.471916,0.769773,0.863636,0.400000,0.166667,0.204455,0.500000,0,1,0,0
3,1,0.525551,0.005935,0.454545,0.628571,0.333333,0.342301,0.833333,0,1,0,0
4,0,0.106919,0.005935,0.454545,0.628571,0.000000,0.155915,0.333333,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
102,1,0.393257,0.805248,0.227273,0.771429,0.166667,0.132405,0.333333,0,1,0,0
103,0,0.191581,0.587597,0.227273,0.028571,0.000000,0.015563,0.333333,0,0,1,0
104,0,0.005769,0.885185,0.227273,0.228571,0.000000,0.963072,0.000000,0,1,0,0
105,0,1.000000,0.791698,0.181818,0.000000,0.000000,0.287067,0.333333,1,0,0,0


In [ ]:
check = [7057, 3972, 9646, 256, 997, 690, 4921, 727, 2729, 9406, 201, 466, 1504, 1672, 1555, 927, 5619, 2595, 3404, 1225, 4117, 6311, 5565, 8544, 4027, 2187, 4382, 4817, 2565, 2334, 3432, 8125, 7676, 1317, 2227, 6204, 963, 930, 8870, 9881, 8443, 8346, 141, 2666, 2353, 7585, 2153, 5042, 2911, 8645, 3539, 158, 8849, 6902, 4822, 254, 6923, 1942, 3433, 7140, 9832, 5649, 8526, 7228, 5097, 5919, 920, 644, 7062, 9791, 7842, 2602, 3800, 5879, 4268, 4873, 8698, 632, 8265, 9069, 9248, 1615, 4314, 5734, 8035, 3247, 6915, 2750, 7690, 2901, 8978, 1914, 5615, 49, 4786, 5231, 8972, 6483, 9273, 7088, 8225, 8772, 5688, 958, 9668, 157, 9157, 2409, 8683, 8101, 8821, 967, 5863, 4492, 2564, 7440, 1694, 3168, 9774, 5822, 1127, 1362, 9641, 4325, 8695, 433, 7751, 7281, 1711, 4494, 7038, 3289, 8573, 6076, 5778, 8333, 6115, 1517, 9491, 4575, 68, 5298, 2743, 4774, 3109, 4032, 3255, 801, 2661, 709, 1918, 4538, 2398, 9163, 4302, 9689, 1261, 7119, 5431, 4761, 1750, 3541, 3590, 7434, 3767, 2154, 3137, 4684, 9982, 9634, 4141, 2165, 8491, 8384, 5696, 50, 9567, 2061, 8850, 1460, 3696, 6978, 1121, 4548, 6400, 4765, 1482, 1828, 2108, 8787, 6084, 8955, 3171, 1960, 8168, 8423, 4714, 4411, 7601, 2303, 7641, 2865, 6784, 8048, 6646, 6732, 2267, 5882, 7176, 4555, 5318, 1124, 1403, 3374, 7237, 8530, 4769, 1288, 2678, 2229, 7149, 5281, 3577, 1827, 720, 2769, 8108, 6591, 4361, 4963, 3942, 5116, 2708, 9277, 4471, 4120, 5946, 3177, 1150, 6535, 5616, 1819, 3037, 4781, 1236, 1341, 9624, 4186, 6514, 9901, 7916, 7438, 204, 3550, 6804, 8768, 121, 6945, 6820, 8251, 2975, 5175, 5651, 7293, 4602, 5168, 1749, 3436, 548, 1326, 1296, 362, 3095, 1874, 868, 4807, 1889, 9591, 9394, 5742, 1324, 9420, 8784, 3959, 8720, 4730, 4480, 6240, 8062, 6205, 7238, 3028, 8264, 9170, 1103, 5152, 5955, 2142, 9387, 6841, 5328, 8704, 3565, 1976, 3415, 2697, 8612, 6523, 9057, 9383, 9601, 6125, 8654, 7613, 5123, 1563, 6062, 519, 6278, 9604, 2193, 4678, 1650, 1335, 9607, 5089, 1519, 3934, 5831, 2510, 6429, 2294, 5355, 8428, 805, 6365, 3364, 6216, 9907, 8170, 4839, 1772, 3200, 7618, 986, 1168, 5382, 7042, 6337, 8842, 3281, 646, 4882, 2973, 415, 824, 6031, 8160, 6590, 9073, 4451, 1849, 2534, 7729, 5743, 5729, 203, 938, 529, 5957, 8984, 7528, 8250, 1921, 4596, 9422, 4095, 5629, 4579, 5411, 1082, 3811, 1484, 2524, 9045, 2372, 3906, 1208, 5701, 231, 4892, 8611, 5332, 5102, 7210, 4326, 9657, 7410, 2111, 8216, 629, 3488, 2787, 6148, 1046, 8029, 9984, 118, 4128, 5201, 3860, 8097, 3124, 9541, 9300, 8282, 1728, 1160, 387, 3886, 8179, 2349, 9326, 44, 5303, 2476, 2172, 2805, 3242, 909, 4214, 7953, 3845, 996, 3141, 9820, 7972, 2408, 3657, 3009, 5041, 9941, 107, 7205, 170, 9704, 5774, 3849, 9099, 48, 5244, 8359, 9235, 5378, 9737, 3011, 7400, 61, 9930, 5306, 6066, 8490, 7816, 7087, 5339, 8447, 3175, 5395, 943, 8702, 4126, 5268, 27, 2470, 8540, 8613, 9324, 8820, 6071, 9012, 8774, 2987, 706, 8203, 5293, 9968, 2794, 8777, 7774, 953, 3078, 9699, 7515, 303, 2631, 9651, 6688, 9060, 5282, 9986, 9971, 159, 2402, 7079, 5413, 8837, 1751, 8666, 6519, 6157, 3306, 9916, 6966, 9361, 7568, 6941, 1230, 889, 5650, 1385, 6631, 6625, 4485, 5190, 2247, 7037, 6614, 7743, 2721, 5529, 2028, 427, 7875, 7130, 9439, 7555, 253, 3355, 1255, 426, 3441, 5631, 8753, 2016, 7014, 762, 3662, 5986, 8138, 7955, 1406, 1804, 948, 1135, 5477, 8636, 4695, 7020, 5952, 5031, 2446, 5757, 7543, 2390, 5389, 905, 8240, 810, 6340, 8116, 2033, 7758, 7463, 2099, 6437, 5816, 3064, 6082, 9336, 4456, 9214, 7097, 8113, 2613, 7673, 2181, 9318, 9244, 9452, 4672, 8310, 4240, 6334, 8688, 8454, 941, 4944, 143, 9520, 236, 6559, 1016, 2587, 3463, 7150, 8857, 2804, 1920, 7669, 2586, 191, 3548, 3901, 5686, 8977, 208, 1829, 3630, 5555, 8200, 4014, 7279, 8090, 5337, 1461, 6410, 6750, 6912, 3735, 114, 5054, 2702, 2912, 7094, 9414, 6118, 798, 6859, 9253, 6202, 515, 5773, 9730, 9752, 3226, 7872, 3620, 3073, 8175, 654, 6899, 2023, 4402, 9215, 6909, 6785, 1567, 4708, 2025, 767, 2996, 9347, 9585, 9143, 412, 3847, 7663, 1736, 7446, 3535, 7620, 8942, 6422, 7476, 4432, 9093, 9694, 4504, 8948, 8212, 2789, 9196, 1649, 8736, 4335, 4918, 6531, 4006, 2947, 2392, 8560, 8236, 451, 3372, 7220, 4658, 1886, 5164, 161, 8572, 4901, 636, 8859, 6423, 7251, 5292, 4160, 8422, 4438, 9294, 425, 6783, 4900, 8322, 7048, 4946, 2603, 7392, 1311, 499, 4898, 2680, 7334, 4732, 7890, 6173, 3296, 5194, 351, 4833, 3265, 6578, 6346, 663, 3440, 2312, 7363, 5912, 9574, 4277, 8805, 1399, 2746, 56, 4893, 8765, 6818, 8172, 714, 6805, 9428, 6771, 4837, 9382, 9882, 305, 5504, 583, 903, 731, 3334, 1515, 6954, 8370, 3530, 3936, 1665, 4525, 7866, 7805, 3725, 8735, 1444, 316, 2704, 5837, 8663, 6609, 328, 5975, 8518, 7101, 3342, 7278, 2734, 7609, 4573, 396, 3576, 2634, 5398, 6351, 4600, 4640, 4857, 8751, 4069, 8987, 4231, 5232, 4644, 8094, 6428, 2466, 7980, 3560, 6673, 2157, 7787, 5479, 9620, 2562, 8312, 4775, 728, 2780, 9908, 2202, 6344, 6774, 9269, 6271, 8458, 4052, 1446, 3963, 9696, 8080, 1617, 4216, 2102, 2443, 7817, 5806, 5271, 8277, 1193, 6128, 8888, 6552, 932, 9332, 8335, 6604, 3579, 8128, 1030, 2393, 5854, 5561, 2490, 3056, 2810, 3983, 2826, 1904, 2391, 4336, 8920, 643, 7418, 5025, 6618, 1518, 2027, 6798, 7989, 4669, 247, 3588, 1884, 2115, 7263, 3708, 3406, 3358, 4263, 4979, 1498, 6897, 8474, 3302, 3536, 7006, 2684, 4118, 7897, 9590, 651, 8889, 7869, 9775, 1896, 4722, 5270, 9513, 9386, 9114, 240, 3469, 6723, 7642, 6714, 1055, 9331, 4028, 1873, 9525, 1041, 1669, 84, 3614, 2428, 5391, 981, 9890, 6489, 4381, 7249, 3300, 1670, 1957, 3365, 7782, 9798, 2472, 6320, 7269, 5578, 2520, 5971, 9251, 9873, 719, 4261, 1187, 2982, 3938, 1865, 5241, 3269, 9857, 9161, 3804, 2823, 2477, 3022, 7706, 9685, 8328, 5881, 7497, 1803, 974, 3797, 6993, 2136, 5896, 6987, 6301, 2896, 3026, 3987, 1808, 8319, 5926, 6085, 5537, 7441, 2279, 176, 7874, 9138, 8884, 9472, 8006, 9822, 7067, 2844, 5379, 7754, 1915, 1731, 4863, 623, 3919, 6391, 6448, 4338, 6765, 3625, 9972, 7677, 1201, 7112, 287, 1197, 5939, 9035, 1361]

In [ ]:
check2 = [1845, 6846, 8188, 7181, 3096, 4639, 4482, 5737, 1867, 70, 8078, 1910, 2189, 2024, 7762, 6793, 1224, 8171, 7394, 9435, 2137, 3518, 9995, 260, 6520, 5838, 3352, 4878, 5714, 8408, 8120, 6751, 8445, 9192, 5071, 2389, 1181, 5130, 4847, 3525, 8900, 1632, 7744, 7947, 6672, 5495, 7353, 5263, 268, 4215, 5373, 1999, 6861, 6286, 5846, 3473, 4442, 221, 7968, 9056, 2575, 5256, 3796, 9108, 5220, 4097, 9506, 3006, 6613, 8481, 4933, 563, 3702, 8072, 9793, 1977, 1492, 6433, 899, 3637, 9140, 4065, 7032, 8226, 4608, 2077, 2001, 5751, 5593, 7900, 2261, 7135, 5509, 2838, 4799, 3646, 7445, 9207, 6455, 342, 4172, 3594, 6175, 9722, 5456, 9565, 5857, 1179, 3350, 3896, 3221, 4207, 5708, 7423, 9209, 1640, 814, 4680, 4068, 8498, 5044, 9515, 7245, 8835, 8005, 3805, 9560,9066, 6736, 1443, 1097, 8450, 8913, 3997, 4566, 8904, 8059, 5368, 5392, 7793, 1814, 4026, 6592, 1246, 3658, 243, 9343, 5618, 7187, 9086, 4917, 6068, 4743, 4636, 6009, 364, 2676, 8382, 6198, 5982, 5851, 2034, 9275, 38, 7050, 7499, 5360, 6986, 2851, 4968, 5476, 5370, 615, 9539, 6108, 4445, 1821, 7680, 5290, 4571, 5347, 1172, 8283, 8197, 1695, 7879, 29, 5055, 1575, 5949, 1855, 6402, 5783, 4450, 7162, 6059, 578, 4541, 4439, 7309, 1837, 285, 2340, 4425, 1701, 7765, 5577, 7218, 1764, 3554, 6968, 145, 1159, 1513, 2338, 3873, 6907, 5731, 5874, 3125, 8906, 6731, 8792, 4199, 1526, 5544, 9861,1832, 5788, 3705, 9019, 8217, 3224, 8959, 3456, 9016, 9046, 6253, 5834, 1470, 4865, 1188, 1487, 224, 3049, 1991, 4122, 6486, 776, 5892, 4871, 3215, 979, 2480, 3437, 8246, 8177, 2696, 4378, 3393, 6189, 5597, 5903, 4278, 3863, 7561, 6803, 4374, 1753, 7174, 6834, 185, 7831, 3462, 861, 7317, 8911, 8694, 5376, 1338, 5084, 5361, 9325, 964, 3864, 6415, 1586, 4063, 9293, 3977, 7519, 3995, 130, 2786, 4130, 1656, 2116, 1100, 732, 6186, 6778, 4895, 4258, 1576, 2224, 9973, 8851, 4697, 6510, 5539, 1250, 2927, 4937, 190, 4540, 1528, 2709, 9136, 3691, 7003, 7739, 5599, 8819, 1061, 7184, 3371, 6156]
len(check2)

327

In [ ]:
n = 100
num = []
while len(num) != n:
    s = random.randint(1,10000)
    if s not in num and s not in check and s not in check2:
        num.append(s)
print(num)

[1832, 5788, 3705, 9019, 8217, 3224, 8959, 3456, 9016, 9046, 6253, 5834, 1470, 4865, 1188, 1487, 224, 3049, 1991, 4122, 6486, 776, 5892, 4871, 3215, 979, 2480, 3437, 8246, 8177, 2696, 4378, 3393, 6189, 5597, 5903, 4278, 3863, 7561, 6803, 4374, 1753, 7174, 6834, 185, 7831, 3462, 861, 7317, 8911, 8694, 5376, 1338, 5084, 5361, 9325, 964, 3864, 6415, 1586, 4063, 9293, 3977, 7519, 3995, 130, 2786, 4130, 1656, 2116, 1100, 732, 6186, 6778, 4895, 4258, 1576, 2224, 9973, 8851, 4697, 6510, 5539, 1250, 2927, 4937, 190, 4540, 1528, 2709, 9136, 3691, 7003, 7739, 5599, 8819, 1061, 7184, 3371, 6156]


In [ ]:
X1 = all1.drop('규모', axis=1)
X1 = np.array(X1)
y1 = all1['규모']

X2 = all2.drop('규모', axis=1)
X2 = np.array(X2)
y2 = all2['규모']

# 표준화

- k-fold, relu
- k-fold, leakyrelu
- 계층적 k-fold, relu
- 계층적 k-fold, leakyrelu

In [ ]:
rep = 2 # No.of replications
acc1=[]
acc2=[]
loss1 = []
loss2 = []
# 모델 저장경로는 model1~8로 고정, 상위 폴더명을 따로 설정
folder = "data2"

for i in range(rep):
    print("\n","iter =",i,"\n")

    #######
    print(" @@@@@@@@@@@@@@@@@@@@ Standard KFold ReLU @@@@@@@@@@@@@@@@@@@@ ")
    kf_acc1, kf_loss1 = [], []
    kf1 = KFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X1.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    ## 경로 꼭 확인하기
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model1/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in kf1.split(X1):
        X_train, X_test = X1[train_index], X1[test_index]
        y_train, y_test = y1[train_index], y1[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        ## 여기도 경로 확인하기! 위에서 저장한 경로랑 같은 곳이어야 함
        best = load_model('/content/drive/MyDrive/'+folder+'/model1/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        kf_acc1.append(scores1[1])
        kf_loss1.append(scores1[0])

    ########
    print(" @@@@@@@@@@@@@@@@@@@@ Standard KFold LeakyReLU @@@@@@@@@@@@@@@@@@@@ ")
    kf_acc2, kf_loss2 = [], []
    kf2 = KFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, input_shape=(X1.shape[1],)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(16))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model2/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in kf2.split(X1):
        X_train, X_test = X1[train_index], X1[test_index]
        y_train, y_test = y1[train_index], y1[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        best = load_model('/content/drive/MyDrive/'+folder+'/model2/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        kf_acc2.append(scores1[1])
        kf_loss2.append(scores1[0])

    ##########
    print(" @@@@@@@@@@@@@@@@@@@@ Standard StratifiedKFold ReLU @@@@@@@@@@@@@@@@@@@@ ")
    skf_acc1, skf_loss1 = [], []
    skf1 = StratifiedKFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X1.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model3/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in skf1.split(X1,y1):
        X_train, X_test = X1[train_index], X1[test_index]
        y_train, y_test = y1[train_index], y1[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        best = load_model('/content/drive/MyDrive/'+folder+'/model3/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        skf_acc1.append(scores1[1])
        skf_loss1.append(scores1[0])

    ###########
    print(" @@@@@@@@@@@@@@@@@@@@ Standard StratifiedKFold LeakyReLU @@@@@@@@@@@@@@@@@@@@ ")
    skf_acc2, skf_loss2 = [], []
    skf2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, input_shape=(X1.shape[1],)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(16))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model4/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in skf2.split(X1,y1):
        X_train, X_test = X1[train_index], X1[test_index]
        y_train, y_test = y1[train_index], y1[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        best = load_model('/content/drive/MyDrive/'+folder+'/model4/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        skf_acc2.append(scores1[1])
        skf_loss2.append(scores1[0])

    ######################################################

    #######
    print(" @@@@@@@@@@@@@@@@@@@@ Min-Max KFold ReLU @@@@@@@@@@@@@@@@@@@@ ")
    kf_acc3, kf_loss3 = [], []
    kf3 = KFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X2.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    ## 경로 꼭 확인하기
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model5/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in kf3.split(X2):
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        ## 여기도 경로 확인하기! 위에서 저장한 경로랑 같은 곳이어야 함
        best = load_model('/content/drive/MyDrive/'+folder+'/model5/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        kf_acc3.append(scores1[1])
        kf_loss3.append(scores1[0])

    ########
    print(" @@@@@@@@@@@@@@@@@@@@ Min-Max KFold LeakyReLU @@@@@@@@@@@@@@@@@@@@ ")
    kf_acc4, kf_loss4 = [], []
    kf4 = KFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, input_shape=(X2.shape[1],)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(16))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model6/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in kf4.split(X2):
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        best = load_model('/content/drive/MyDrive/'+folder+'/model6/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        kf_acc4.append(scores1[1])
        kf_loss4.append(scores1[0])

    ##########
    print(" @@@@@@@@@@@@@@@@@@@@ Min-Max StratifiedKFold ReLU @@@@@@@@@@@@@@@@@@@@ ")
    skf_acc3, skf_loss3 = [], []
    skf3 = StratifiedKFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X2.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model7/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in skf3.split(X2,y2):
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        best = load_model('/content/drive/MyDrive/'+folder+'/model7/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        skf_acc3.append(scores1[1])
        skf_loss3.append(scores1[0])

    ###########
    print(" @@@@@@@@@@@@@@@@@@@@ Min-Max StratifiedKFold LeakyReLU @@@@@@@@@@@@@@@@@@@@ ")
    skf_acc4, skf_loss4 = [], []
    skf4 = StratifiedKFold(n_splits=5, shuffle=True, random_state=num[i])
    model = Sequential()
    model.add(Dense(32, input_shape=(X1.shape[1],)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(16))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001), metrics=['acc'])

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20, min_delta = 0.001)
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.95, verbose = 0, min_lr = 0)
    mc = ModelCheckpoint('/content/drive/MyDrive/'+folder+'/model8/best_model'+str(i)+'.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

    for train_index, test_index in skf4.split(X2,y2):
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        # 학습 및 예측
        history = model.fit(X_train, y_train, epochs = 2000, batch_size = 1,
        verbose = 0, validation_split = 0.2, callbacks=[earlystop,reducelr,mc])
        best = load_model('/content/drive/MyDrive/'+folder+'/model8/best_model'+str(i)+'.h5')
        scores1 = best.evaluate(X_test, y_test)
        skf_acc4.append(scores1[1])
        skf_loss4.append(scores1[0])

    acc1.append([num[i], np.mean(kf_acc1), np.mean(kf_acc2), np.mean(skf_acc1), np.mean(skf_acc2)])
    loss1.append([num[i], np.mean(kf_loss1), np.mean(kf_loss2), np.mean(skf_loss1), np.mean(skf_loss2)])
    acc2.append([num[i], np.mean(kf_acc3), np.mean(kf_acc4), np.mean(skf_acc3), np.mean(skf_acc4)])
    loss2.append([num[i], np.mean(kf_loss3), np.mean(kf_loss4), np.mean(skf_loss3), np.mean(skf_loss4)])

    ######################################################

    result_acc1 = pd.DataFrame(acc1, columns=['seed','k-r','k-l','s-r','s-l'])
    result_acc2 = pd.DataFrame(acc2, columns=['seed','k-r','k-l','s-r','s-l'])
    result_loss1 = pd.DataFrame(loss1, columns=['seed','k-r','k-l','s-r','s-l'])
    result_loss2 = pd.DataFrame(loss2, columns=['seed','k-r','k-l','s-r','s-l'])

    result_acc1.to_excel("/content/drive/MyDrive/"+folder+"/result_acc1.xlsx")
    result_acc2.to_excel("/content/drive/MyDrive/"+folder+"/result_acc2.xlsx")
    result_loss1.to_excel("/content/drive/MyDrive/"+folder+"/result_loss1.xlsx")
    result_loss2.to_excel("/content/drive/MyDrive/"+folder+"/result_loss2.xlsx")

    print("\n","SAVE ", i,"\n")

    ######################################################


 iter = 0 

 @@@@@@@@@@@@@@@@@@@@ Standard KFold ReLU @@@@@@@@@@@@@@@@@@@@ 


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 215ms/step - loss: 0.7718 - acc: 0.2727


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 190ms/step - loss: 0.4489 - acc: 0.8182


...

1/1 [==============================] - 0s 221ms/step - loss: 0.6531 - acc: 0.6190
 @@@@@@@@@@@@@@@@@@@@ Min-Max StratifiedKFold LeakyReLU @@@@@@@@@@@@@@@@@@@@ 


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 144ms/step - loss: 0.6502 - acc: 0.7273


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 148ms/step - loss: 0.4030 - acc: 0.8095

 SAVE  1 



In [ ]:
result_acc1.describe()

In [ ]:
result_acc2.describe()

In [ ]:
result_loss1.describe()

In [ ]:
result_loss2.describe()